## Decision Trees for Classification

In this notebook, we'll use a **Decision Tree Classifier** to predict penguin species based on physical characteristics using the **Palmer Penguins** dataset.

We are using... as input features and Penguin Species as the target feature.

### 1. Imports and Setup

In [3]:
from palmerpenguins import load_penguins
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

/Users/sg2147/Documents/SummerSchool_DT/DecisionTrees_SummerSchool25/venv/lib/python3.12/site-packages/palmerpenguins/penguins.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### 2. Load and Inspect Data

In [ ]:
# Load dataset and drop any nan records
penguins = load_penguins().dropna()

# Preprocess the data
le = preprocessing.LabelEncoder()

# Apply encoding to the categorical data
penguins['encoded'] = le.fit_transform(penguins['species'])

In [ ]:
colours = {'Adelie':'#8966a3','Chinstrap':'#dba162','Gentoo':'#4e7e82',}

In [ ]:
x = penguins[[
            'bill_length_mm',
            'bill_depth_mm',
            # 'flipper_length_mm',
            # 'body_mass_g'
            ]
             ].to_numpy()
y = penguins['encoded'].to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

In [ ]:
for species in list(colours.keys()):
    mask = penguins[penguins['species'] == species]
    plt.scatter(mask['bill_length_mm'], mask['bill_depth_mm'],
                c=colours[species], s=64, label=species)

## Load and Train Decision Tree for Classification

## Visualise the Decision Tree